In [1]:
import os
import sys
import logging
import datetime
import dill as pickle
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
sys.path.insert(1, r'methods')

from data_methods import getData, standardize
# from dimensionality_reduction import reduce_dimensionality,reduce_with
# from beamSearch import EMM, as_string
# from adjPysubgroup import adjustedBestFirstSearch, adjustedDFS, adjustedApriori
# from qualityMeasures import calc_result_bs, calc_result_ps
from interpretabilityMeasures import Feature_Correlation_Scores, DBI_beam, DBI_ps

In [3]:
# Loading the object using dill
sd_results_path = r'C:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\sd_results'
file_name = '-data-reductions.pkl'
with open(os.path.join(sd_results_path,file_name), 'rb') as f:
    loaded_dict =pickle.load(f)

In [11]:
datasets = []
for dataset in dataset:
    file_name = '-data-reductions.pkl'
        en_methods = ['auto_encoder', 'PCA', 'SPCA']
        for en_method in en_methods:
            fcs, fcss = Feature_Correlation_Scores(loaded_dict['vanilla'], loaded_dict[en_method])
            print(en_method, "    ", "\n FCS: ", round(fcs, 2), "    FCSS", round(fcss, 2))

auto_encoder      
 FCS:  0.53     FCSS 0.13
PCA      
 FCS:  0.65     FCSS 0.14
SPCA      
 FCS:  0.72     FCSS 0.19


In [6]:
loaded_dict['auto_encoder']

,target,cat1,cat2,cat3,cat4,cat5
0,1,-1.085433,-0.008109,2.769221,-1.309007,1.818651
1,0,-0.220155,1.821277,0.997081,-0.944791,5.082852
2,1,-0.546220,-0.090474,3.039274,-0.793125,0.585284
3,0,-0.532459,-1.948231,-1.751228,-0.621276,-2.728531
4,1,-0.665896,0.597152,1.729603,-1.021183,2.745559
...,...,...,...,...,...,...
346,1,-0.319230,-0.128999,4.123497,-0.468865,-0.377496
347,1,-0.094618,-0.203330,4.084755,-0.663449,-0.544531
348,1,-0.141873,-0.188322,4.018230,-0.705673,-0.517522
349,1,-0.402674,-0.219408,3.848034,-0.950649,-0.291891


In [21]:
sd_results_path = r'C:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\sd_results'
file_name = '-data-reductions.pkl'
with open(os.path.join(sd_results_path,file_name), 'rb') as f:
    loaded_dict =pickle.load(f)

vanilla
auto_encoder
PCA
SPCA


In [11]:
df_en = pd.DataFrame(loaded_dict)

ValueError: If using all scalar values, you must pass an index